In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('data/reddit.csv')

display(data.head())

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [3]:
data.shape

(37249, 2)

In [4]:
data.sample()['clean_comment'].values

array(['not allowed ask any clarifications have doubt akshully are allowed make fun randians and randimods otherwise '],
      dtype=object)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37249 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37149 non-null  object
 1   category       37249 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 582.1+ KB


In [6]:
data.dropna(inplace=True)

In [7]:
data.drop_duplicates(inplace=True)


In [8]:
data[(data['clean_comment'].str.strip() == '')]

,clean_comment,category
181,,0
4432,\n,0
10592,,0
16173,,0
32149,\n,0
34959,,0


In [9]:
data = data[~(data['clean_comment'].str.strip() == '')]

In [10]:
data['clean_comment'] = data['clean_comment'].str.lower()
data.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [11]:
data[data['clean_comment'].apply(lambda x: x.endswith(' ') or x.startswith(' '))]

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1
...,...,...
37241,let the janta decide not ulema clerics,0
37242,hona hai same with vaccination education insu...,0
37246,downvote karna tha par upvote hogaya,0
37247,haha nice,1


In [12]:
print(data['clean_comment'].apply(lambda x: x.endswith(' ') or x.startswith(' ')).sum())

data['clean_comment'] = data['clean_comment'].str.strip()

print(data['clean_comment'].apply(lambda x: x.endswith(' ') or x.startswith(' ')).sum())

32266
0
